In [1]:
# need to match the system-wide installed cuda-11 for deepspeed to compile
# so install the matching pytorch

!pip install torch==1.7.1+cu110  -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 63.9MB/s eta 0:00:06tcmalloc: large alloc 1147494400 bytes == 0x559d60534000 @  0x7fa4a80ce615 0x559d27c3b06c 0x559d27d1aeba 0x559d27c3de8d 0x559d27d2f99d 0x559d27cb1fe9 0x559d27cacb0e 0x559d27c3f77a 0x559d27cb1e50 0x559d27cacb0e 0x559d27c3f77a 0x559d27cae86a 0x559d27d307c6 0x559d27cadee2 0x559d27d307c6 0x559d27cadee2 0x559d27d307c6 0x559d27cadee2 0x559d27d307c6 0x559d27db2431 0x559d27d13049 0x559d27c7dc84 0x559d27c3e8e9 0x559d27cb2ade 0x559d27c3f69a 0x559d27cada45 0x559d27cace0d 0x559d27c3f77a 0x559d27cada45 0x559d27c3f69a 0x559d27cada45
     |█████████████████████████████▏  | 1055.7MB 1.2MB/s eta 0:01:25tcmalloc: large alloc 1434370048 bytes == 0x559da4b8a000 @  0x7fa4a80ce615 0x559d27c3b06c 0x559d27d1aeba 0x559d27c3de8d 0x559d27d2f99d 0x559d27cb1fe9 0x559d27cacb0e 0x559d27c3f77a 0x559d27cb1e50 0x559d27cacb0e 0x559d27c3f77a 0x559d27cae86a 0x559d27d307c6 0x559d27cadee2 

In [2]:
!pip install deepspeed

     |████████████████████████████████| 286kB 6.3MB/s 
     |████████████████████████████████| 225kB 9.0MB/s 
     |████████████████████████████████| 112kB 8.0MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.3.10-cp37-none-any.whl size=272625 sha256=452186cc24c832b2193de6b7148e2ebd11b03ac7caba3a941fbfe6ba54faf81d
  Stored in directory: /root/.cache/pip/wheels/a3/3c/9c/39a16330874a2c55f61fe2c501e120258975d509177ffdcda7
Successfully built deepspeed


In [3]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install -e .
pip install -r examples/_tests_requirements.txt

Obtaining file:///content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f99cc83a832b9a8908f8735131521efc88df980b222806beffc870378b1332a0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Running setup.py develop for transformers
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=f8051ac777411c02eb0ba0d14e5f1a58c92dffa00417fbb9fa3dae7e390636ac
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for fire: filename=fire-0.4.0-p

Cloning into 'transformers'...
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.


In [4]:
%%bash

cd transformers

cat <<'EOT' > ds_config.json
{
    "fp16": {
        "enabled": true,
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "zero_optimization": {
        "stage": 2,
       "allgather_partitions": true,
       "allgather_bucket_size": 2e8,
       "reduce_scatter": true,
       "reduce_bucket_size": 2e8,
        "overlap_comm": true,
        "contiguous_gradients": true,
        "cpu_offload": true
    },

    "optimizer": {
        "type": "Adam",
        "params": {
            "adam_w_mode": true,
            "lr": 3e-5,
            "betas": [ 0.9, 0.999 ],
            "eps": 1e-8,
            "weight_decay": 3e-7
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": 0,
            "warmup_max_lr": 3e-5,
            "warmup_num_steps": 500
        }
    }
}
EOT


In [5]:
!ls -l

total 8
drwxr-xr-x  1 root root 4096 Mar  1 14:35 sample_data
drwxr-xr-x 15 root root 4096 Mar  4 16:30 transformers


In [6]:

!cd transformers; export BS=16; rm -r output_dir; PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/seq2seq/run_seq2seq.py --model_name_or_path google/mt5-small \
--output_dir output_dir --adam_eps 1e-06  --evaluation_strategy=steps  --do_train --label_smoothing 0.1 --learning_rate 3e-5 --logging_first_step --logging_steps 1000 \
--max_source_length 128 --max_target_length 128 --num_train_epochs 1 --overwrite_output_dir  --per_device_train_batch_size $BS --predict_with_generate --sortish_sampler \
--val_max_target_length 128 --warmup_steps 500 --max_train_samples 2000 --max_val_samples 500 \
--task translation_en_to_ro  --dataset_name wmt16 --dataset_config ro-en --source_prefix "translate English to Romanian: " --deepspeed  ds_config.json --fp16


rm: cannot remove 'output_dir': No such file or directory
[2021-03-04 16:30:56,136] [WARNING] [runner.py:117:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2021-03-04 16:30:56,153] [INFO] [runner.py:355:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 examples/seq2seq/run_seq2seq.py --model_name_or_path google/mt5-small --output_dir output_dir --adam_eps 1e-06 --evaluation_strategy=steps --do_train --label_smoothing 0.1 --learning_rate 3e-5 --logging_first_step --logging_steps 1000 --max_source_length 128 --max_target_length 128 --num_train_epochs 1 --overwrite_output_dir --per_device_train_batch_size 16 --predict_with_generate --sortish_sampler --val_max_target_length 128 --warmup_steps 500 --max_train_samples 2000 --max_val_samples 500 --task translation_en_to_ro --dataset_name wmt16 --dataset_config ro-en --source_prefix translate English to 